In [1]:
pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!wget -q --show-progress -O subs.csv https://www.dropbox.com/scl/fi/x4k7vhf6pze0z5w333573/Subscribers.csv?rlkey=lkntddg23yktg1hdgbgt8nlcg&dl=0

subs.csv            100%[===================>] 125.29M  45.9MB/s    in 2.7s    


In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('subs.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 51 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   V1           300000 non-null  float64
 1   V2           300000 non-null  float64
 2   V3           300000 non-null  float64
 3   V4           300000 non-null  float64
 4   V5           300000 non-null  float64
 5   V6           300000 non-null  float64
 6   V7           300000 non-null  float64
 7   V8           300000 non-null  float64
 8   V9           300000 non-null  float64
 9   V10          300000 non-null  float64
 10  V11          300000 non-null  float64
 11  V12          300000 non-null  float64
 12  V13          300000 non-null  float64
 13  V14          300000 non-null  float64
 14  V15          300000 non-null  float64
 15  V16          300000 non-null  float64
 16  V17          300000 non-null  float64
 17  V18          300000 non-null  float64
 18  V19          300000 non-

In [6]:
x = data.drop(columns=['Subscribers']).values
y = data['Subscribers']

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size =0.2, random_state=32)

In [9]:
scaler = StandardScaler()

x_train_sd = scaler.fit_transform(x_train)
x_test_sd = scaler.transform(x_test)

In [10]:
y_train.value_counts()

,count
Subscribers,
0,198837
1,41163


In [11]:
198837 + 41163

240000

In [12]:
240000 /(2*198837)

0.6035094072028848

In [13]:
240000 /(2*41163)

2.915239414036878

In [14]:
import torch

In [15]:
cw = np.array([0.603, 2.915])

In [16]:
cw_tensor = torch.tensor(cw, dtype=torch.float32)

In [17]:
from torch.utils.data import Dataset, DataLoader # DataLoaders helps in chunking of data i.e. random chunks of data

In [18]:
# len , getitems

class TabularData (Dataset):
  def __init__(self, x,y):
    self.x = torch.tensor(x, dtype=torch.float32)
    self.y = torch.tensor(y.values, dtype=torch.float32)

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

train_ds = TabularData(x_train_sd, y_train)
test_ds = TabularData(x_test_sd, y_test)

In [19]:
train_loader = DataLoader(train_ds, batch_size=10000, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

In [21]:
import torch.nn as nn

In [22]:
class DeepFFN(nn.Module):
  def __init__(self, input_dim):
    # we could customise this further
    # by providing the ability to have a customr number of layers and nodes
    super(DeepFFN, self).__init__()
    self.fc1 = nn.Linear(input_dim, 30)
    self.bn1 = nn.BatchNorm1d(30)
    self.dp1 = nn.Dropout(0.2)

    self.fc2 = nn.Linear(30, 10)
    self.bn2 = nn.BatchNorm1d(10)
    self.dp2 = nn.Dropout(0.2)

    self.fc3 = nn.Linear(10, 1)
    self.bn3 = nn.BatchNorm1d(1)
    self.dp3 = nn.Dropout(0.2)

    self.out = nn.Linear(10,1)

  def forward(self, x):
    x = self.dp1(self.bn1(F.relu(self.fc1(x))))
    x = self.dp2F.relu(self.fc2(x))
    x = self.dp3F.relu(self.fc3(x))
    x = self.out(x)
    return x
